In [1]:
# Import modules
import pandas as pd
import numpy as np

In [4]:
df = pd.read_parquet('https://github.com/Immobilienrechner-Challenge/data/blob/main/kaggle_uncleaned.parquet?raw=true')

In [9]:
df.shape

(24556, 132)

In [6]:
df_cleaned = pd.DataFrame()

In [7]:
df_cleaned['living_space'] = df['Space extracted']

In [8]:
df_cleaned['living_space'].count()

11563

In [10]:
## Rooms
df_cleaned['rooms'] = df['details_structured'].str.extract('(\d+\.?\d?) rooms').astype(float)
df_cleaned['rooms'] = df['rooms'].fillna(df['No. of rooms:'])

In [12]:
df_cleaned['rooms'] = df_cleaned['rooms'].replace('', np.nan)

In [13]:
df_cleaned['rooms'].count()

23557

In [21]:
## Plot Area
df_cleaned['plot_area'] = df['Plot_area_merged'].fillna(df['detail_responsive#surface_property']).fillna(df['Land area:'])

In [22]:
df_cleaned['plot_area'].count()

9484

In [24]:
## Floor Space
df_cleaned['floor_space'] = df['Floor_space_merged'].fillna(df['detail_responsive#surface_usable']).fillna(df['Floor space:'])

In [25]:
## Floor
df_cleaned['floor'] = df['Floor_merged'].fillna(df['detail_responsive#floor']).fillna(df['Floor'])

In [26]:
## Availability
df_cleaned['availability'] = df['Availability_merged'].fillna(df['detail_responsive#available_from'])

In [29]:
## zip code
df_cleaned['zip_code'] = df['address'].str.extract(r'(\d{4}) [A-ZÀ-Ÿ]') 
df_cleaned['zip_address_s'] = df['address_s'].str.extract("(\d{4}) [A-ZÀ-Ÿ]")
df_cleaned['zip_code'] = df_cleaned['zip_code'].fillna(df_cleaned['zip_address_s']) 

In [30]:
df_cleaned['zip_code'].count()

24555

In [33]:
df.iloc[df_cleaned[df_cleaned['zip_code'].isna()].index]

,Unnamed: 0.1,Unnamed: 0,Municipality,Living space,Plot area,Floor space,Availability,location,description,detailed_description,...,Floor space:,Number of floors:,Volume:,plz,Number of toilets:,Gross yield:,Minimum floor space:,space_cleaned,Type:,Hall height:
22022,22032,22032,rancate,None,None,None,None,6862 rancate,Questo grazioso appartamento totalmente ristru...,"Description\n""Grazioso appartamento nel cuore ...",...,None,NaN,None,6862.0,NaN,None,None,NaN,Apartment,None


In [34]:
df_cleaned.dropna(subset=['zip_code'], inplace=True)

In [35]:
# clean up typos
df_cleaned.loc[df_cleaned['zip_code'] == '2737', 'zip_code'] = '2735'
df_cleaned.loc[df_cleaned['zip_code'] == '3217', 'zip_code'] = '3127'
df_cleaned.loc[df_cleaned['zip_code'] == '3364', 'zip_code'] = '3365'

In [37]:
df_cleaned['municipality'] = df['address'].str.extract(r'\d{4} (.+?),')
df_cleaned['municipality_address_s'] = df['address_s'].str.extract(r"\d{4} (.+)$")
df_cleaned['municipality'] = df_cleaned['municipality'].fillna(df_cleaned['municipality_address_s'])

In [38]:
df_xlsx_plz = pd.read_excel("../../eda/utils/plz.xlsx", sheet_name='Blatt1')
df_xlsx_plz.drop(['Kanton', 'Canton', 'Cantone', 'Land', 'Pays', 'Paese'], axis=1, inplace=True)
df_xlsx_plz.rename(columns={df_xlsx_plz.columns[0]: 'plz', df_xlsx_plz.columns[1]: 'municipality', df_xlsx_plz.columns[2]: 'canton'}, inplace=True)
df_xlsx_plz.drop_duplicates(subset=['plz'], inplace=True)

/opt/conda/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [40]:
df_cleaned['canton'] = df_cleaned['zip_code'].astype(int).map(df_xlsx_plz.set_index('plz')['canton']).copy()

In [42]:
# Clean up cantons
df_cleaned.loc[df_cleaned['zip_code'] == '1919', 'canton'] = 'VS'
df_cleaned.loc[df_cleaned['zip_code'] == '1818', 'canton'] = 'VD'

In [44]:
## Street
df_cleaned["street"] = df['address'].str.extract(r"(.+), \d{4}")
df_cleaned.loc[df_cleaned['street'] == '-', 'street'] = np.nan
df_cleaned.loc[df_cleaned['street'] == 'à', 'street'] = np.nan

In [45]:
df_cleaned["street_nr"] = df_cleaned['street'].str.extract(r'^.+ (\d.+)')
      
# separate street name from street number
df_cleaned["street"] = df_cleaned['street'].str.extract(r'^(.+?) \d')
df_cleaned["street"] = df_cleaned['street'].str.rstrip()